### Workshop: DLT Homework

### Question-1. dlt version

In [ ]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [2]:
!pip freeze | findstr dlt

dlt==1.13.0


In [3]:
import dlt 

print(dlt.__version__)

1.13.0


This is `1.13.0` the version of dlt that was installed!

### Question 2. dlt pipeline

#### dlt resource

For reading the FAQ data, we have this helper function.

In [4]:
import requests
import dlt

@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

dlt pipeline 

Let's create pipeline. 

We need to define a destination for that. Let's use the `qdrant` one.

In [5]:
# destination 
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In this case, we tell `dlt` and Qdrant to create a folder with our data, and the name for it will be `db.qdrant`

Let's run it:

In [7]:
# pipeline 
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"
)

load_info = pipeline.run(zoomcamp_data())

print(pipeline.last_trace)

p:\Projects\LLM-ZoomCamp\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]p:\Projects\LLM-ZoomCamp\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Karthik\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to 

Run started at 2025-07-11 10:21:31.197235+00:00 and COMPLETED in 21.28 seconds with 4 steps.
Step extract COMPLETED in 0.99 seconds.

Load package 1752229300.66538 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.23 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752229300.66538 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 10.62 seconds.
Pipeline zoomcamp_pipeline load step completed in 10.58 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used p:\Projects\LLM-ZoomCamp\workshops\dlt\db.qdrant location to store data
Load package 1752229300.66538 is LOADED and contains no failed jobs

Step run COMPLETED in 21.28 seconds.
Pipeline zoomcamp_pipeline load step completed in 10.58 seconds
1 load package(s) were loaded to desti

948 rows were inserted into the `zoomcamp_data` collection.

### Question 3. Embedding

In `meta.json` file, we see that dlt used this embedding model: `fast-bge-small-en`. This model's dimensionality is 384